In [1]:
import numpy as np
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="6"
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [3]:
import PIL
import tensorflow as tf
import random
import re
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
#disable_eager_execution()
enable_eager_execution()

In [4]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import argparse
from tensorflow.keras.layers import Input
import scipy.misc

In [5]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.applications.resnet50 import ResNet50

In [6]:
import time

In [22]:
from tensorflow.keras import backend as K

In [23]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

In [7]:
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

In [8]:
x_train = np.expand_dims(x_train, axis=-1)

# [optional]: we may need 3 channel (instead of 1)
x_train = np.repeat(x_train, 3, axis=-1)

# it's always better to normalize 
x_train = x_train.astype('float32')

In [9]:
# resize the input shape , i.e. old shape: 28, new shape: 32
x_train = tf.image.resize(x_train, [32,32]) # if we want to resize 

In [10]:
input = tf.keras.Input(shape=(32,32,3))
efnet = tf.keras.applications.ResNet50(weights='imagenet',
                                             include_top = False, 
                                             input_tensor = input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()(efnet.output)

# Finally, we add a classification layer.
output = tf.keras.layers.Dense(10, activation='softmax', use_bias=True)(gap)

# bind all

In [11]:
model_ = tf.keras.Model(input, output)
q_model = tfmot.quantization.keras.quantize_model(model_)

In [12]:
model = tf.keras.Model(input, output)

In [13]:
q_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
quantize_layer (QuantizeLayer)  (None, 32, 32, 3)    3           input_1[0][0]                    
__________________________________________________________________________________________________
quant_conv1_pad (QuantizeWrappe (None, 38, 38, 3)    1           quantize_layer[1][0]             
__________________________________________________________________________________________________
quant_conv1_conv (QuantizeWrapp (None, 16, 16, 64)   9601        quant_conv1_pad[0][0]            
______________________________________________________________________________________________

In [14]:
model.load_weights("../../weights/resnet_mnist_fp.h5")
q_model.load_weights("../../weights/resnet_mnist_q.h5")
model.trainable = False
q_model.trainable = False
model.compile()
q_model.compile()
preprocess = tf.keras.applications.resnet.preprocess_input

In [20]:
def second(image,label):
    orig_image = tf.identity(image)
    input_image = tf.identity(image)
    orig_logist = tf.identity(model.predict(preprocess(input_image)[None,...]))
    orig_label =  np.argmax(orig_logist[0])

    
    quant_logist = tf.identity(q_model.predict(preprocess(input_image)[None,...]))
    quant_label =  np.argmax(quant_logist[0])

    
    if orig_label != quant_label:
        return -2,-2,-2,-2,-2
    
    A = 0
    start_time = time.time()
    for iters in range(0,grad_iterations):
        with tf.GradientTape() as g:
            g.watch(input_image)
            loss1 = K.mean(model(preprocess(input_image+A)[None,...], training = False)[..., orig_label])
            loss2 = K.mean(q_model(preprocess(input_image+A)[None,...], training = False)[..., orig_label])
            final_loss = K.mean(loss1 - c*loss2)


        grads = normalize(g.gradient(final_loss, input_image))
        A += tf.sign(grads) * step
        A = tf.clip_by_value(A, -epsilon, epsilon)
        test_image_deprocess = tf.clip_by_value(input_image + A, 0, 255)
        test_image = preprocess(test_image_deprocess)[None,...]
        pred1, pred2= model.predict(test_image), q_model.predict(test_image)
        label1, label2 = np.argmax(pred1[0]), np.argmax(pred2[0])
        
        if not label1 == label2:
            if label1 == orig_label:
        
                total_time = time.time() - start_time
                
                norm = np.max(np.abs(A))
                
                return total_time, norm, iters, test_image_deprocess, A
            
    gen_img_deprocessed = tf.clip_by_value(orig_image + A, 0, 255)

    return -1, -1, -1, gen_img_deprocessed, A

In [25]:
def calc_normal_success(method, x, y, folderName='', filterName='',dataName='',dataFolder='',locald = ''):
    
    total=0
    count=0
    badimg = 0
    
    timeStore = []
    advdistStore = []
    stepsStore = []
    
    timeStorek = []
    advdistStorek = []
    stepsStorek = []
    
    for i, image in enumerate(x):
        label = y[i]
        if label != 0:
            continue

        time, advdist, steps, gen, A = method(image,label)

        total += 1
        
        if time > -2:
            np.save(locald+'/'+folderName+"/"+dataName+str(total)+".npy", gen)
            np.save(locald+'/'+filterName+"/"+dataName+str(total)+".npy", A)

        if time == -1:
            print("Didnt find anything")
            continue

        if time == -2:
            badimg += 1
            total -= 1
            print("Bad Image",badimg)
            continue

        if time == -3:
            badimg += 1
            total -= 1
            print("Incorrect Image",badimg)
            continue
            
        count += 1

        print("Number seen:",total)
        print("No. worked:", count)

In [17]:
BATCH_SIZE = 50
c = 1
grad_iterations = 20
step = 1
epsilon = 8
net='res'

In [ ]:
calc_normal_success(second,x_train,y_train,
                   folderName=net + 'net_mnist_images_second', filterName=net +'net_mnist_filters_second',dataName='second', dataFolder=net +'net_mnist_data_second', locald ='./results')